In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [ ]:
# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

In [ ]:
torch.manual_seed(1337)

In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

--2024-07-23 11:22:36--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.02s   

2024-07-23 11:22:37 (49.6 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string


In [ ]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [ ]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [ ]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

In [ ]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [ ]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [ ]:
class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [ ]:
model = GPTLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

0.209729 M parameters


In [ ]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [ ]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.1959, val loss 4.1962
step 100: train loss 2.6229, val loss 2.6166
step 200: train loss 2.4555, val loss 2.4488
step 300: train loss 2.3810, val loss 2.3928
step 400: train loss 2.3202, val loss 2.3223
step 500: train loss 2.2364, val loss 2.2541
step 600: train loss 2.1812, val loss 2.2234
step 700: train loss 2.1326, val loss 2.1583
step 800: train loss 2.0932, val loss 2.1352
step 900: train loss 2.0499, val loss 2.0987
step 1000: train loss 2.0349, val loss 2.0819
step 1100: train loss 1.9994, val loss 2.0706
step 1200: train loss 1.9857, val loss 2.0726
step 1300: train loss 1.9638, val loss 2.0401
step 1400: train loss 1.9386, val loss 2.0296
step 1500: train loss 1.9028, val loss 1.9947
step 1600: train loss 1.8821, val loss 1.9914
step 1700: train loss 1.8785, val loss 1.9813
step 1800: train loss 1.8746, val loss 1.9878
step 1900: train loss 1.8451, val loss 1.9599
step 2000: train loss 1.8294, val loss 1.9525
step 2100: train loss 1.8333, val loss 1.9716


In [ ]:
def generate(idx):
  idx_cond = idx[:, -block_size:]
  # get the predictions
  logits, loss = model(idx_cond)
  # focus only on the last time step
  logits = logits[:, -1, :] # becomes (B, C)
  # apply softmax to get probabilities
  probs = F.softmax(logits, dim=-1) # (B, C)
  # sample from the distribution
  idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
  # append sampled index to the running sequence
  idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
  return idx

In [ ]:
context = "w"
print(encode(context))

[61]


In [ ]:
idx=torch.tensor([encode(context)])
idx

tensor([[61]])

In [ ]:
context = "w"
idx=torch.tensor([encode(context)], dtype=torch.long, device=device)
print(decode(m.generate(idx, max_new_tokens=500)[0].tolist()))

with pleason mary.
You with a what
That hen of-renions the men.

CLRIFFHORD:
Where no sugh own.

Jurious Moy:
Which might. 
CEMIO:
Pray it therefultianus well.

LEONd Seved Made inkingitings:
Poin that the gRiction:
Whith I shall now then is dande,
To may I wantoblaing againsting wear so! why lumps a made so lack.

Prive, must Lord,
And I it have brother'nd Captey
Though to meised the nowness futing appriantuss
Beieng itter us speiclies kings as him
Fravous crawatch time wout in for Glouck.

GLOU


In [ ]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))
#open('more.txt', 'w').write(decode(m.generate(context, max_new_tokens=10000)[0].tolist()))

In [ ]:
context = "w"
idx=torch.tensor([encode(context)], dtype=torch.long, device=device)
logits, loss = model(idx)
logits

tensor([[[  0.4301,   2.8255,  -0.9732,  -7.4144,  -7.1434,  -0.2701,   0.7377,
           -1.3133,  -0.0407,  -7.0788,  -0.9276,  -0.3784,  -0.7852,  -2.5938,
           -3.9895,  -5.6384,  -4.8989,  -4.9770,  -4.9688,  -6.7367,  -3.2602,
           -2.0530,  -6.4898,  -4.8292,  -3.8396,  -5.8515,  -5.9446,  -4.4851,
           -6.5002,  -6.7461,  -4.1959,  -5.8436,  -4.0003,  -5.6313,  -7.2865,
           -3.7865, -10.2564,  -5.5001,  -5.3904,   3.5384,  -1.1425,  -0.1845,
           -1.0654,   3.5278,  -0.2386,  -1.3817,   3.8595,   4.0593,  -4.5906,
           -2.6691,   0.9871,  -1.7440,   1.1630,   3.2278,  -1.7138,  -3.7100,
            1.1601,   0.7361,   0.5997,  -1.4390,  -2.7474,  -1.5636,  -3.4233,
            0.5334,  -3.7831]]], device='cuda:0', grad_fn=<ViewBackward0>)

In [ ]:
# focus only on the last time step
logits = logits[:, -1, :] # becomes (B, C)
# apply softmax to get probabilities
probs = F.softmax(logits, dim=-1) # (B, C)
# sample from the distribution
idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
idx_next

tensor([[43]], device='cuda:0')

In [ ]:
import time
while True:
    query=input("\nyou ")
    if query == "exit":
        break
    idx = torch.tensor([encode(query)], dtype=torch.long, device=device)
    logits,_ = model(idx)
    # focus only on the last time step
    logits = logits[:, -1, :] # becomes (B, C)
    # apply softmax to get probabilities
    probs = F.softmax(logits, dim=-1) # (B, C)
    # sample from the distribution
    idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
    outputs=decode(m.generate(idx_next, max_new_tokens=50)[0].tolist())
    print("Generating Text")
    for item in outputs:
      print("{:<0}".format(item), end="")
      time.sleep(0.3)

print("Thank you")


you tiny
Generating Text
:
Ay, and have drues whe, my north wind,
I wastoo'd
you exit
Thank you


In [ ]:
my_list = ["apple", "banana", "cherry"]

print(" ".join(my_list))

apple banana cherry


In [ ]:
for item in my_list:
  print("{:<1}".format(item), end=" ")

apple banana cherry 

In [ ]:
import torch
import torch.nn as nn

# Sample outputs (replace with your actual outputs)
output1 = torch.randn(1024, 768)
output2 = torch.randn(1024, 768)

# Concatenate along the desired dimension (here, dimension 1)
concatenated_output = torch.cat((output1, output2), dim=1)  # Output shape: (1024, 1536)

# Define the linear layer
linear_layer = nn.Linear(1536, output_size)  # Replace output_size with desired output dimension

# Feed the concatenated output into the linear layer
final_output = linear_layer(concatenated_output)

NameError: name 'output_size' is not defined

In [ ]:
import re

original_string = """3:03 AM Session 1084791523 (Full Name: Melissa Keim) has been started by the system...
3:03 AM Connecting to: control.rsc-app26-01.logmeinrescue.com (158.120.24.6:443)
3:03 AM Connected to Applet (Elliptic Curve Diffie-Hellman 2048 bits, ECDHE-RSA-AES256-GCM-SHA384 256 bits)
3:03 AM Shrivallabha: Welcome to Arise Live Support, I will be glad to assist you. Please hold for a moment while I review your issue details. During this time is there anything you would like to add regarding the issue you are experiencing?"""

# Define a regular expression to match the unwanted part
pattern = r".*?\.+\n\n"  # Matches everything until two consecutive newlines

# Use re.sub to substitute the matched part with an empty string
new_string = re.sub(pattern, "", original_string)

print(new_string)

3:03 AM Session 1084791523 (Full Name: Melissa Keim) has been started by the system...
3:03 AM Connecting to: control.rsc-app26-01.logmeinrescue.com (158.120.24.6:443)
3:03 AM Connected to Applet (Elliptic Curve Diffie-Hellman 2048 bits, ECDHE-RSA-AES256-GCM-SHA384 256 bits)
3:03 AM Shrivallabha: Welcome to Arise Live Support, I will be glad to assist you. Please hold for a moment while I review your issue details. During this time is there anything you would like to add regarding the issue you are experiencing?
